In [2]:
"""
Generating ewt data for different cycles for all levels

N4 F1

"""

'\nGenerating ewt data for different cycles for all levels\n\nN4 F1\n\n'

In [3]:
import pandas as pd
import numpy as np
import pywt
import matplotlib.pyplot as plt
import os
import glob
import re

C:\Users\vinee\anaconda3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [4]:
import numpy as np
#%EWT functions
def ewt_vineet(f,boundaries):
    # signal spectrum
    ff = np.fft.fft(f)
    ff = abs(ff[0:int(np.ceil(ff.size / 2))])  # one-sided magnitude

    #extend the signal by mirroring to deal with boundaries
    ltemp = int(np.ceil(f.size/2)) #to behave the same as matlab's round
    fMirr =  np.append(np.flip(f[0:ltemp-1],axis = 0),f)
    fMirr = np.append(fMirr,np.flip(f[-ltemp-1:-1],axis = 0))
    ffMirr = np.fft.fft(fMirr)
    # build the corresponding filter bank
    mfb = EWT_Meyer_FilterBank(boundaries, ffMirr.size)

    # filter the signal to extract each subband
    ewt = np.zeros(mfb.shape)
    for k in range(mfb.shape[1]):
        ewt[:, k] = np.real(np.fft.ifft(np.conjugate(mfb[:, k]) * ffMirr))
    ewt = ewt[ltemp - 1:-ltemp, :]

    return ewt


def EWT_Meyer_FilterBank(boundaries, Nsig):
    """
     =========================================================================
     function mfb=EWT_Meyer_FilterBank(boundaries,Nsig)

     This function generate the filter bank (scaling function + wavelets)
     corresponding to the provided set of frequency segments

     Input parameters:
       -boundaries: vector containing the boundaries of frequency segments (0
                    and pi must NOT be in this vector)
       -Nsig: signal length

     Output:
       -mfb: cell containing each filter (in the Fourier domain), the scaling
             function comes first and then the successive wavelets

     Author: Jerome Gilles
     Institution: UCLA - Department of Mathematics
     Year: 2012
     Version: 1.0

     Python Version: Vinícius Rezende Carvalho - vrcarva@ufmg.br
     Universidade Federal de Minas Gerais - Brasil
     Núcleo de Neurociências
     =========================================================================
     """
    Npic = len(boundaries)
    # compute gamma
    gamma = 1
    for k in range(Npic - 1):
        r = (boundaries[k + 1] - boundaries[k]) / (boundaries[k + 1] + boundaries[k])
        if r < gamma:
            gamma = r
    r = (np.pi - boundaries[Npic - 1]) / (np.pi + boundaries[Npic - 1])
    if r < gamma:
        gamma = r
    gamma = (1 - 1 / Nsig) * gamma  # this ensure that gamma is chosen as strictly less than the min

    mfb = np.zeros([Nsig, Npic + 1])

    # EWT_Meyer_Scaling
    Mi = int(np.floor(Nsig / 2))
    w = np.fft.fftshift(np.linspace(0, 2 * np.pi - 2 * np.pi / Nsig, num=Nsig))
    w[0:Mi] = -2 * np.pi + w[0:Mi]
    aw = abs(w)
    yms = np.zeros(Nsig)
    an = 1. / (2 * gamma * boundaries[0])
    pbn = (1. + gamma) * boundaries[0]
    mbn = (1. - gamma) * boundaries[0]
    for k in range(Nsig):
        if aw[k] <= mbn:
            yms[k] = 1
        elif ((aw[k] >= mbn) and (aw[k] <= pbn)):
            yms[k] = np.cos(np.pi * EWT_beta(an * (aw[k] - mbn)) / 2)
    yms = np.fft.ifftshift(yms)
    mfb[:, 0] = yms

    # generate rest of the wavelets
    for k in range(Npic - 1):
        mfb[:, k + 1] = EWT_Meyer_Wavelet(boundaries[k], boundaries[k + 1], gamma, Nsig)

    mfb[:, Npic] = EWT_Meyer_Wavelet(boundaries[Npic - 1], np.pi, gamma, Nsig)

    return mfb
def EWT_beta(x):
    """
    Beta = EWT_beta(x)
    function used in the construction of Meyer's wavelet
    """
    if x<0:
        bm=0
    elif x>1:
        bm=1
    else:
        bm=(x**4)*(35.-84.*x+70.*(x**2)-20.*(x**3))
    return bm
def EWT_Meyer_Wavelet(wn,wm,gamma,Nsig):
    """
    =========================================================
    ymw=EWT_Meyer_Wavelet(wn,wm,gamma,N)
    
    Generate the 1D Meyer wavelet in the Fourier
    domain associated to scale segment [wn,wm] 
    with transition ratio gamma
    
    Input parameters:
      -wn : lower boundary
      -wm : upper boundary
      -gamma : transition ratio
      -N : number of point in the vector
    
    Output:
      -ymw: Fourier transform of the wavelet on the band [wn,wm]
    Author: Jerome Gilles
    Institution: UCLA - Department of Mathematics
    Year: 2012
    Version: 1.0
    
    Python Version: Vinícius Rezende Carvalho - vrcarva@ufmg.br
    Universidade Federal de Minas Gerais - Brasil
    Núcleo de Neurociências 
    ==========================================================            
    """
    Mi=int(np.floor(Nsig/2))
    w=np.fft.fftshift(np.linspace(0,2*np.pi - 2*np.pi/Nsig,num = Nsig))
    w[0:Mi]=-2*np.pi+w[0:Mi]
    aw=abs(w)
    ymw=np.zeros(Nsig)
    an=1./(2*gamma*wn)
    am=1./(2*gamma*wm)
    pbn=(1.+gamma)*wn
    mbn=(1.-gamma)*wn
    pbm=(1.+gamma)*wm
    mbm=(1.-gamma)*wm

    for k in range(Nsig):
        if ((aw[k]>=pbn) and (aw[k]<=mbm)):
            ymw[k]=1
        elif ((aw[k]>=mbm) and (aw[k]<=pbm)):
            ymw[k]=np.cos(np.pi*EWT_beta(am*(aw[k]-mbm))/2)
        elif ((aw[k]>=mbn) and (aw[k]<=pbn)):
            ymw[k]=np.sin(np.pi*EWT_beta(an*(aw[k]-mbn))/2)
    ymw=np.fft.ifftshift(ymw)
    return ymw

<h4>Boundaries to be used for EWT Computation will be the average boundaries obtained  from N1R1

In [5]:
boundaries = np.array([0.63679854007417,1.24043747184364,1.90051054875325,2.19052315291591,2.42542041587323,2.62882317400665,2.76529989452493,2.88996614389811,3.01463240497312])

In [6]:
pwd

'C:\\Users\\vinee\\Documents\\BTP'

In [10]:
cd resampled_dataset/

C:\Users\vinee\Documents\BTP\resampled_dataset_f1_n1\resampled_dataset


In [23]:
cd ..

C:\Users\vinee\Documents\BTP\resampled_dataset_f1_n1


In [11]:
def dataset(folder,boundaries,loudness_data,x_train,y_train):
    os.chdir(folder)
    my_files = glob.glob('*txt')
    for r in range(1,4,1):
        for j in range(1,1001,1):
            idx = []
            for i in range(len(my_files)):
                string1 = my_files[i]
                p = list(map(int, re.findall(r'\d+', string1)))
                if p[-2] == j and p[-3] == r and int(p[-1])>40:
                    idx.append(i)
            res = np.zeros((12,10,126))
            i  = 0
            for m in idx:
                file = np.loadtxt(my_files[m])
                ewt = ewt_vineet(file,boundaries)
                ewt = ewt.transpose()
                res[i] = ewt
                i+=1
            x_train.append(res)
            y_train.append(loudness_data)
    return x_train,y_train

In [12]:
pwd

'C:\\Users\\vinee\\Documents\\BTP\\resampled_dataset_f1_n1\\resampled_dataset'

In [13]:
cd ..

C:\Users\vinee\Documents\BTP\resampled_dataset_f1_n1


In [28]:
x_train = []
y_train = []

In [15]:
loudness_data_n1 = np.array([0.203489,0.461973,0.421456,0.525431,0.691075,0.733513,0.761766,1.076224,1.245589,1.340460,1.642738,1.773085])

In [25]:
x_train,y_train = dataset('resampled_dataset',boundaries,loudness_data_n1,x_train,y_train)

In [29]:
len(x_train)

0

In [32]:
my_files = glob.glob('*txt')

In [33]:
len(my_files)

131000

In [35]:
for j in range(1,1001,1):
    idx = []
    for i in range(len(my_files)):
        string1 = my_files[i]
        p = list(map(int, re.findall(r'\d+', string1)))
        if p[-2] == j and p[-3] == 1 and int(p[-1])>40:
            idx.append(i)
    res = np.zeros((12,10,126))
    i  = 0
    for m in idx:
        file = np.loadtxt(my_files[m])
        ewt = ewt_vineet(file,boundaries)
        ewt = ewt.transpose()
        res[i] = ewt
        i+=1
    x_train.append(res)
    y_train.append(loudness_data_n1)

In [40]:
for j in range(1,1001,1):
    idx = []
    for i in range(len(my_files)):
        string1 = my_files[i]
        p = list(map(int, re.findall(r'\d+', string1)))
        if p[-2] == j and p[-3] == 2 and int(p[-1])>40:
            idx.append(i)
    res = np.zeros((12,10,126))
    i  = 0
    for m in idx:
        file = np.loadtxt(my_files[m])
        ewt = ewt_vineet(file,boundaries)
        ewt = ewt.transpose()
        res[i] = ewt
        i+=1
    x_train.append(res)
    y_train.append(loudness_data_n1)

In [41]:
for j in range(1,1001,1):
    idx = []
    for i in range(len(my_files)):
        string1 = my_files[i]
        p = list(map(int, re.findall(r'\d+', string1)))
        if p[-2] == j and p[-3] == 3 and int(p[-1])>40:
            idx.append(i)
    res = np.zeros((12,10,126))
    i  = 0
    for m in idx:
        file = np.loadtxt(my_files[m])
        ewt = ewt_vineet(file,boundaries)
        ewt = ewt.transpose()
        res[i] = ewt
        i+=1
    x_train.append(res)
    y_train.append(loudness_data_n1)

In [44]:
x = np.array(x_train)
y = np.array(y_train)

In [43]:
len(y_train)

3000

In [45]:
cd ..

C:\Users\vinee\Documents\BTP\resampled_dataset_f1_n1


In [46]:
cd ..

C:\Users\vinee\Documents\BTP


In [52]:
os.chdir('resampled_dataset_f1_n2/')

In [53]:
my_files = glob.glob('*txt')

In [54]:
len(my_files)

101000

In [55]:
loudness_data_n2 = np.array([-0.767364,-0.476870,-0.193201,0.078374,0.172603,0.419829,0.500441,0.720262,0.920215,1.080807,1.295526,1.501556])


In [56]:
for j in range(1,1001,1):
    idx = []
    for i in range(len(my_files)):
        string1 = my_files[i]
        p = list(map(int, re.findall(r'\d+', string1)))
        if p[-2] == j and p[-3] == 1 and int(p[-1])>40:
            idx.append(i)
    res = np.zeros((12,10,126))
    i  = 0
    for m in idx:
        file = np.loadtxt(my_files[m])
        ewt = ewt_vineet(file,boundaries)
        ewt = ewt.transpose()
        res[i] = ewt
        i+=1
    x_train.append(res)
    y_train.append(loudness_data_n2)

In [57]:
len(x_train)

4000

In [58]:
for j in range(1,1001,1):
    idx = []
    for i in range(len(my_files)):
        string1 = my_files[i]
        p = list(map(int, re.findall(r'\d+', string1)))
        if p[-2] == j and p[-3] == 2 and int(p[-1])>40:
            idx.append(i)
    res = np.zeros((12,10,126))
    i  = 0
    for m in idx:
        file = np.loadtxt(my_files[m])
        ewt = ewt_vineet(file,boundaries)
        ewt = ewt.transpose()
        res[i] = ewt
        i+=1
    x_train.append(res)
    y_train.append(loudness_data_n2)

In [61]:
len(x_train)

6000

In [60]:
for j in range(1,1001,1):
    idx = []
    for i in range(len(my_files)):
        string1 = my_files[i]
        p = list(map(int, re.findall(r'\d+', string1)))
        if p[-2] == j and p[-3] == 3 and int(p[-1])>40:
            idx.append(i)
    res = np.zeros((12,10,126))
    i  = 0
    for m in idx:
        file = np.loadtxt(my_files[m])
        ewt = ewt_vineet(file,boundaries)
        ewt = ewt.transpose()
        res[i] = ewt
        i+=1
    x_train.append(res)
    y_train.append(loudness_data_n2)

In [63]:
cd ..

C:\Users\vinee\Documents\BTP\resampled_dataset_f1_n2


In [64]:
cd ..

C:\Users\vinee\Documents\BTP


In [65]:
os.chdir('resampled_dataset_f1_n4/')

In [66]:
my_files = glob.glob('*txt')

In [67]:
len(my_files)

93000

In [12]:
loudness_data_n4 = np.array([0.502216,0.559076,0.654683,0.955243,0.968337,0.885020,0.965291,1.290606,1.397530,1.460671,1.729242,1.781090])


In [70]:
x = np.array(x_train)

In [71]:
y = np.array(y_train)

In [72]:
x.shape

(6000, 12, 10, 126)

In [73]:
y.shape

(6000, 12)

In [74]:
for j in range(1,1001,1):
    idx = []
    for i in range(len(my_files)):
        string1 = my_files[i]
        p = list(map(int, re.findall(r'\d+', string1)))
        if p[-2] == j and p[-3] == 1 and int(p[-1])>40:
            idx.append(i)
    res = np.zeros((12,10,126))
    i  = 0
    for m in idx:
        file = np.loadtxt(my_files[m])
        ewt = ewt_vineet(file,boundaries)
        ewt = ewt.transpose()
        res[i] = ewt
        i+=1
    x_train.append(res)
    y_train.append(loudness_data_n4)

In [75]:
for j in range(1,1001,1):
    idx = []
    for i in range(len(my_files)):
        string1 = my_files[i]
        p = list(map(int, re.findall(r'\d+', string1)))
        if p[-2] == j and p[-3] == 2 and int(p[-1])>40:
            idx.append(i)
    res = np.zeros((12,10,126))
    i  = 0
    for m in idx:
        file = np.loadtxt(my_files[m])
        ewt = ewt_vineet(file,boundaries)
        ewt = ewt.transpose()
        res[i] = ewt
        i+=1
    x_train.append(res)
    y_train.append(loudness_data_n4)

In [76]:
for j in range(1,1001,1):
    idx = []
    for i in range(len(my_files)):
        string1 = my_files[i]
        p = list(map(int, re.findall(r'\d+', string1)))
        if p[-2] == j and p[-3] == 3 and int(p[-1])>40:
            idx.append(i)
    res = np.zeros((12,10,126))
    i  = 0
    for m in idx:
        file = np.loadtxt(my_files[m])
        ewt = ewt_vineet(file,boundaries)
        ewt = ewt.transpose()
        res[i] = ewt
        i+=1
    x_train.append(res)
    y_train.append(loudness_data_n4)

In [77]:
len(x_train)

9000

In [78]:
len(y_train)

9000

In [79]:
cd ..

C:\Users\vinee\Documents\BTP


In [80]:
x = np.array(x_train)
y = np.array(y_train)

<ipython-input-80-2e813cd87558>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  y = np.array(y_train)


In [81]:
x.shape

(9000, 12, 10, 126)

In [82]:
y.shape

(9000,)

In [84]:
len(y_train[8998])

14

In [85]:
y[0]

array([0.203489, 0.461973, 0.421456, 0.525431, 0.691075, 0.733513,
       0.761766, 1.076224, 1.245589, 1.34046 , 1.642738, 1.773085])

In [86]:
len(y[0])

12

In [88]:
len(y[5999])

12

In [90]:
len(loudness_data_n4)

12

In [91]:
len(loudness_data_n1)

12

In [92]:
len(loudness_data_n2)

12

In [93]:
for i in range(5999,9000):
    y_train[i] = loudness_data_n4

In [94]:
y = np.array(y_train)

In [95]:
y.shape

(9000, 12)

In [96]:
x.shape

(9000, 12, 10, 126)

In [97]:
pwd

'C:\\Users\\vinee\\Documents\\BTP'

In [98]:
os.chdir('resampled_dataset_f1_n5/')

In [99]:
my_files=  glob.glob('*txt')

In [100]:
len(my_files)

109000

In [20]:
loudness_data_n5 = np.array([0.776163,0.859717,0.739199,1.114550,1.283773,1.146815,1.214429,1.605921,1.819364,1.927476,2.032690,2.031693])


In [106]:
len(loudness_data_n5)

12

In [107]:
for j in range(1,1001,1):
    idx = []
    for i in range(len(my_files)):
        string1 = my_files[i]
        p = list(map(int, re.findall(r'\d+', string1)))
        if p[-2] == j and p[-3] == 1 and int(p[-1])>40:
            idx.append(i)
    res = np.zeros((12,10,126))
    i  = 0
    for m in idx:
        file = np.loadtxt(my_files[m])
        ewt = ewt_vineet(file,boundaries)
        ewt = ewt.transpose()
        res[i] = ewt
        i+=1
    x_train.append(res)
    y_train.append(loudness_data_n5)

In [108]:
for j in range(1,1001,1):
    idx = []
    for i in range(len(my_files)):
        string1 = my_files[i]
        p = list(map(int, re.findall(r'\d+', string1)))
        if p[-2] == j and p[-3] == 2 and int(p[-1])>40:
            idx.append(i)
    res = np.zeros((12,10,126))
    i  = 0
    for m in idx:
        file = np.loadtxt(my_files[m])
        ewt = ewt_vineet(file,boundaries)
        ewt = ewt.transpose()
        res[i] = ewt
        i+=1
    x_train.append(res)
    y_train.append(loudness_data_n5)

In [109]:
for j in range(1,1001,1):
    idx = []
    for i in range(len(my_files)):
        string1 = my_files[i]
        p = list(map(int, re.findall(r'\d+', string1)))
        if p[-2] == j and p[-3] == 3 and int(p[-1])>40:
            idx.append(i)
    res = np.zeros((12,10,126))
    i  = 0
    for m in idx:
        file = np.loadtxt(my_files[m])
        ewt = ewt_vineet(file,boundaries)
        ewt = ewt.transpose()
        res[i] = ewt
        i+=1
    x_train.append(res)
    y_train.append(loudness_data_n5)

In [110]:
x = np.array(x_train)

In [111]:
y = np.array(y_train)

In [112]:
print(x.shape)
print(y.shape)

(12000, 12, 10, 126)
(12000, 12)


In [114]:
os.chdir('resampled_dataset_f1_n7/')

In [116]:
my_files = glob.glob('*txt')

In [117]:
len(my_files)

120000

In [120]:
loudness_data_n7 = np.array([0.265712,0.627094,0.724554,1.015205,1.132958,1.725353,1.697663,1.814931,1.891386,2.309499,2.465781,2.630860])


In [121]:
len(loudness_data_n7)

12

In [122]:
for j in range(1,1001,1):
    idx = []
    for i in range(len(my_files)):
        string1 = my_files[i]
        p = list(map(int, re.findall(r'\d+', string1)))
        if p[-2] == j and p[-3] == 1 and int(p[-1])>40:
            idx.append(i)
    res = np.zeros((12,10,126))
    i  = 0
    for m in idx:
        file = np.loadtxt(my_files[m])
        ewt = ewt_vineet(file,boundaries)
        ewt = ewt.transpose()
        res[i] = ewt
        i+=1
    x_train.append(res)
    y_train.append(loudness_data_n7)

In [123]:
for j in range(1,1001,1):
    idx = []
    for i in range(len(my_files)):
        string1 = my_files[i]
        p = list(map(int, re.findall(r'\d+', string1)))
        if p[-2] == j and p[-3] == 2 and int(p[-1])>40:
            idx.append(i)
    res = np.zeros((12,10,126))
    i  = 0
    for m in idx:
        file = np.loadtxt(my_files[m])
        ewt = ewt_vineet(file,boundaries)
        ewt = ewt.transpose()
        res[i] = ewt
        i+=1
    x_train.append(res)
    y_train.append(loudness_data_n7)

In [124]:
for j in range(1,1001,1):
    idx = []
    for i in range(len(my_files)):
        string1 = my_files[i]
        p = list(map(int, re.findall(r'\d+', string1)))
        if p[-2] == j and p[-3] == 3 and int(p[-1])>40:
            idx.append(i)
    res = np.zeros((12,10,126))
    i  = 0
    for m in idx:
        file = np.loadtxt(my_files[m])
        ewt = ewt_vineet(file,boundaries)
        ewt = ewt.transpose()
        res[i] = ewt
        i+=1
    x_train.append(res)
    y_train.append(loudness_data_n7)

In [125]:
len(x_train)

15000

In [126]:
len(y_train)

15000

In [127]:
x = np.array(x_train)

In [128]:
y = np.array(y_train)

In [132]:
print(x.shape,y.shape,sep = ' = x Shape , Y shape = ')

(15000, 12, 10, 126) = x Shape , Y shape = (15000, 12)


In [133]:
# done for n1,n2,n4,n5,n7

C:\Users\vinee\Documents\BTP


In [134]:
os.chdir('resampled_dataset_f1_n6/')

In [135]:
my_files = glob.glob('*txt')

In [136]:
len(my_files)

150000

In [140]:
loudness_data_n6 = np.array([-0.602983,-0.203698,0.110290,0.531146,0.989981,1.062504,1.391716,1.478321,1.598163,1.779723,2.019546,2.297004])


In [141]:
len(loudness_data_n6)

12

In [142]:
for j in range(1,1001,1):
    idx = []
    for i in range(len(my_files)):
        string1 = my_files[i]
        p = list(map(int, re.findall(r'\d+', string1)))
        if p[-2] == j and p[-3] == 1 and int(p[-1])>40:
            idx.append(i)
    res = np.zeros((12,10,126))
    i  = 0
    for m in idx:
        file = np.loadtxt(my_files[m])
        ewt = ewt_vineet(file,boundaries)
        ewt = ewt.transpose()
        res[i] = ewt
        i+=1
    x_train.append(res)
    y_train.append(loudness_data_n6)

In [143]:
for j in range(1,1001,1):
    idx = []
    for i in range(len(my_files)):
        string1 = my_files[i]
        p = list(map(int, re.findall(r'\d+', string1)))
        if p[-2] == j and p[-3] == 2 and int(p[-1])>40:
            idx.append(i)
    res = np.zeros((12,10,126))
    i  = 0
    for m in idx:
        file = np.loadtxt(my_files[m])
        ewt = ewt_vineet(file,boundaries)
        ewt = ewt.transpose()
        res[i] = ewt
        i+=1
    x_train.append(res)
    y_train.append(loudness_data_n6)

In [144]:
for j in range(1,1001,1):
    idx = []
    for i in range(len(my_files)):
        string1 = my_files[i]
        p = list(map(int, re.findall(r'\d+', string1)))
        if p[-2] == j and p[-3] == 3 and int(p[-1])>40:
            idx.append(i)
    res = np.zeros((12,10,126))
    i  = 0
    for m in idx:
        file = np.loadtxt(my_files[m])
        ewt = ewt_vineet(file,boundaries)
        ewt = ewt.transpose()
        res[i] = ewt
        i+=1
    x_train.append(res)
    y_train.append(loudness_data_n6)

In [145]:
print(len(x_train),len(y_train))

18000 18000


In [146]:
x = np.array(x_train)
y = np.array(y_train)

In [147]:
# done for n1,n2,n4,n5,n7,n6

In [148]:
cd ..

C:\Users\vinee\Documents\BTP


In [150]:
os.chdir('resampled_dataset_f1_n3/')

In [153]:
my_files = glob.glob('*txt')

In [154]:
len(my_files)

39000

In [157]:
loudness_data_n3 = np.array([-0.365368,-0.002425,0.104951,0.144388,0.318761,0.540428,0.655482, 0.994505, 0.934209,1.060224, 1.293006,1.501077])


In [158]:
len(loudness_data_n3)

12

In [159]:
for j in range(1,1001,1):
    idx = []
    for i in range(len(my_files)):
        string1 = my_files[i]
        p = list(map(int, re.findall(r'\d+', string1)))
        if p[-2] == j and p[-3] == 1 and int(p[-1])>40:
            idx.append(i)
    res = np.zeros((12,10,126))
    i  = 0
    for m in idx:
        file = np.loadtxt(my_files[m])
        ewt = ewt_vineet(file,boundaries)
        ewt = ewt.transpose()
        res[i] = ewt
        i+=1
    x_train.append(res)
    y_train.append(loudness_data_n3)

In [160]:
for j in range(1,1001,1):
    idx = []
    for i in range(len(my_files)):
        string1 = my_files[i]
        p = list(map(int, re.findall(r'\d+', string1)))
        if p[-2] == j and p[-3] == 2 and int(p[-1])>40:
            idx.append(i)
    res = np.zeros((12,10,126))
    i  = 0
    for m in idx:
        file = np.loadtxt(my_files[m])
        ewt = ewt_vineet(file,boundaries)
        ewt = ewt.transpose()
        res[i] = ewt
        i+=1
    x_train.append(res)
    y_train.append(loudness_data_n3)

In [161]:
for j in range(1,1001,1):
    idx = []
    for i in range(len(my_files)):
        string1 = my_files[i]
        p = list(map(int, re.findall(r'\d+', string1)))
        if p[-2] == j and p[-3] == 3 and int(p[-1])>40:
            idx.append(i)
    res = np.zeros((12,10,126))
    i  = 0
    for m in idx:
        file = np.loadtxt(my_files[m])
        ewt = ewt_vineet(file,boundaries)
        ewt = ewt.transpose()
        res[i] = ewt
        i+=1
    x_train.append(res)
    y_train.append(loudness_data_n3)

In [162]:
print(len(x_train),len(y_train))

21000 21000


In [163]:
x = np.array(x_train)

In [164]:
y = np.array(y_train)

In [165]:
np.save('C:/Users/vinee/Documents/BTP/complete_x',x)
np.save('C:/Users/vinee/Documents/BTP/complete_y',y)

In [1]:
# dataset creation for Checking model not seeing N1

In [8]:
x_train = []
y_train = []

In [9]:
os.chdir('resampled_dataset_f1_n4/')

In [27]:
my_files = glob.glob('*txt')

In [11]:
len(my_files)

93000

In [13]:
for j in range(1,1001,1):
    idx = []
    for i in range(len(my_files)):
        string1 = my_files[i]
        p = list(map(int, re.findall(r'\d+', string1)))
        if p[-2] == j and p[-3] == 4 and int(p[-1])>40:
            idx.append(i)
    res = np.zeros((12,10,126))
    i  = 0
    for m in idx:
        file = np.loadtxt(my_files[m])
        ewt = ewt_vineet(file,boundaries)
        ewt = ewt.transpose()
        res[i] = ewt
        i+=1
    x_train.append(res)
    y_train.append(loudness_data_n4)

In [14]:
len(x_train)

1000

In [15]:
x = np.array(x_train)
y = np.array(y_train)

In [16]:
np.save('C:/Users/vinee/Documents/BTP/x_new_n4',x)
np.save('C:/Users/vinee/Documents/BTP/y_new_n4',y)

In [17]:
cd ..

C:\Users\vinee\Documents\BTP


In [18]:
x_train = []
y_train = []

In [19]:
os.chdir('resampled_dataset_f1_n5/')

In [21]:
my_files = glob.glob('*txt')

In [22]:
for j in range(1,1001,1):
    idx = []
    for i in range(len(my_files)):
        string1 = my_files[i]
        p = list(map(int, re.findall(r'\d+', string1)))
        if p[-2] == j and p[-3] == 4 and int(p[-1])>40:
            idx.append(i)
    res = np.zeros((12,10,126))
    i  = 0
    for m in idx:
        if p[-1]
        file = np.loadtxt(my_files[m])
        ewt = ewt_vineet(file,boundaries)
        ewt = ewt.transpose()
        res[i] = ewt
        i+=1
    x_train.append(res)
    y_train.append(loudness_data_n5)

KeyboardInterrupt: 

In [26]:
cd resampled_dataset_f1_n4/

C:\Users\vinee\Documents\BTP\resampled_dataset_f1_n4


In [28]:
idx = []
for i in range(len(my_files)):
    string1 = my_files[i]
    p = list(map(int, re.findall(r'\d+', string1)))
    if p[-2] == j and p[-3] == 4 and int(p[-1])>40:
        idx.append(i)

In [29]:
idx

[55128, 55132, 55133, 55134, 55135, 55136, 55137, 55138, 55139]

In [30]:
len(idx)

9

In [35]:
my_files[idx[-1]]

'F1_raw_N4_R4_cycle_860_80.txt'